In [ ]:
# --- Cell 0: Interactive File Setup ---
%matplotlib widget

from pathlib import Path

import ipywidgets as widgets
from ipyfilechooser import FileChooser

import opym

print("1. Select your base OME-TIF file (e.g., ...Pos0.ome.tif).")
print("2. Select the Timepoint (T) you want to use for the MIP.")
print("3. Run Cell 1 to calculate the MIP.")

# --- Create the file chooser ---
file_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    filter_pattern="*.ome.tif",
    title="<b>Select Base OME-TIF File:</b>",
)

# --- Slider for Timepoint selection ---
t_slider = widgets.IntSlider(
    description="Timepoint (T):",
    min=0,
    max=199,  # Will be updated by Cell 1
    step=1,
    value=0,
)

display(file_chooser, t_slider)

In [ ]:
# --- Cell 1: Create the MIPs ---
# This cell now calls the `create_mip` function from your package

try:
    file_to_inspect = Path(file_chooser.value)
    t_index = t_slider.value

    # Call the refactored function
    mip_data, vmin, vmax, lazy_data, t_max = opym.create_mip(file_to_inspect, t_index)

    # Update the slider in Cell 0 with the correct max T
    t_slider.max = t_max
    t_slider.value = t_index

    print("\nYou can now run Cell 2 to select ROIs.")

except FileNotFoundError:
    print("❌ ERROR: File not found. Please select a file in Cell 0 and re-run.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 2: Select ROIs Interactively ---
# This cell uses the new ROISelector class from your package

try:
    # This will display the plot and wait for you to draw two ROIs
    selector = opym.interactive_roi_selector(mip_data, vmin, vmax)
except NameError:
    print("❌ ERROR: 'mip_data' not found. Please run Cell 1 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 3: Get ROIs and Auto-Align ---
# This cell gets the ROIs from the selector and calls the align function

try:
    # This call will block until two ROIs are drawn in the plot from Cell 2
    unaligned_rois = selector.get_rois()

    top_roi_unaligned = unaligned_rois[0]
    bottom_roi_unaligned = unaligned_rois[1]

    print("\n--- Auto-Aligning ROIs ---")
    # Call the refactored alignment function
    bottom_roi_aligned = opym.align_rois(
        lazy_data, top_roi_unaligned, bottom_roi_unaligned, t_index=t_slider.value
    )

    # Store the final, aligned ROIs for processing
    top_roi = top_roi_unaligned
    bottom_roi = bottom_roi_aligned

    print("\n✅ Alignment complete. Run Cell 4 to visualize.")

except NameError:
    print(
        "❌ ERROR: 'selector' or 'lazy_data' not found. Please run Cells 1 and 2 first."
    )
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 4: Visualize Final Alignment ---
# This cell calls the new visualization function from your package

try:
    opym.visualize_alignment(mip_data, top_roi, bottom_roi, vmin, vmax)
    print("\nRun Cell 5 to save these ROIs to your log file.")
except NameError:
    print("❌ ERROR: Required variables not found. Please run Cells 1, 2, and 3 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 5: Save ROIs to Log File ---
# This cell uses the save_rois_to_log function from your package

try:
    # Derive paths to find the log file location
    # (We just need the output_log path, so format doesn't matter)
    paths = opym.derive_paths(file_to_inspect, opym.OutputFormat.ZARR)

    opym.save_rois_to_log(
        log_file=paths.output_log,
        base_file=file_to_inspect,  # <-- FIX: Corrected parameter name and value
        top_roi=top_roi,
        bottom_roi=bottom_roi,
    )

    print(f"✅ ROIs saved to {paths.output_log.name}")
    print("You can now use these ROIs for batch processing via the CLI.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run all previous cells.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")